## Installations

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install contextualSpellCheck

## Imports

In [ ]:
import spacy
import contextualSpellCheck
import csv

### Models

In [ ]:
nlp = spacy.load("en_core_web_sm")
contextualSpellCheck.add_to_pipe(nlp)

## Function Definitions

In [ ]:
def spell_check(text):
    doc = nlp(text)

    return doc._.performed_spellCheck, doc._.outcome_spellCheck

In [ ]:
def correct_spellings(csv_file, out_csv_file, column_name, should_log=False):
    # read a csv file, in each line, see if spellCheck was performed(if text was correctly spelled) and print original and corrected texts
    lines = []

    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        lines = list(reader)

        # Index of the column to be spell checked
        column_index = lines[0].index(column_name)

        # For each line, perform spell check
        for i in range(1, len(lines)):
            line = lines[i][column_index]
            performed, outcome = spell_check(line)

            # If spell check was performed, update the line and print the original and corrected texts
            if should_log and performed:
                print(line)
                print(outcome)
                print()

                # Update the line
                lines[i][column_index] = outcome

    with open(out_csv_file, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(lines)

    print("Spell check completed")

In [ ]:
def pre_process(csv_file, out_csv_file, column_name):
    # read a csv file, in each line, remove characters apart from alphanumerics and whitespaces in the given column
    lines = []

    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        lines = list(reader)

        # Index of the column to be pre processed
        column_index = lines[0].index(column_name)

        for i in range(1, len(lines)):
            lines[i][column_index] = ''.join([c for c in lines[i][column_index] if c.isalnum() or c.isspace()])

    # write the pre-processed data to the resulting csv file
    with open(out_csv_file, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(lines)

    print("Pre procesing completed")

## Main

The CSV file locations need to be changed as applicable if being run locally.

In [ ]:
docs_file = "/content/Query_Doc/docs.csv"
queries_file = "/content/Query_Doc/queries.csv"
pre_processed_docs_file = "/content/Query_Doc/pre_processed_docs.csv"
pre_processed_queries_file = "/content/Query_Doc/pre_processed_queries.csv"
spell_checked_docs_file = "/content/Query_Doc/spell_checked_docs.csv"
spell_checked_queries_file = "/content/Query_Doc/spell_checked_queries.csv"

Pre processing

In [ ]:
pre_process(docs_file, pre_processed_docs_file, "doc_text")
pre_process(queries_file, pre_processed_queries_file, "query_text")

Spell correction on the pre processed docs and queries

In [ ]:
# Print logs(original and corrected for query doc only)
correct_spellings(pre_processed_queries_file, spell_checked_queries_file, "query_text", True)
correct_spellings(pre_processed_docs_file, spell_checked_docs_file, "doc_text")